<a href="https://colab.research.google.com/github/MadhulikaPutrevu2/breakhis/blob/master/RESNET50BREAKHIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install q keras==2.3.1

     |████████████████████████████████| 377 kB 13.7 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires keras~=2.6, but you have keras 2.3.1 which is incompatible.


In [ ]:
%tensorflow_version 2.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.0`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_applications
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adamax, SGD, RMSprop
from keras.layers import Flatten, Input, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model
from keras.applications import ResNet50
from keras.layers.core import Dropout, Dense
from keras.callbacks import EarlyStopping
from keras.models import Model, load_model
from keras.initializers import he_normal, glorot_normal
from keras import optimizers
from keras.regularizers import l2
import os
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fid = drive.ListFile({'q':"title='Data.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Data.zip')

In [ ]:
f.keys()

In [ ]:
!unzip Data.zip


In [ ]:
PATH = '/content/Data'

In [ ]:
os.listdir(PATH)

In [ ]:
train_dir = os.path.join(PATH,
                         'train')
test_dir = os.path.join(PATH,
                        'validation')

In [ ]:
train_benign_dir = os.path.join(train_dir,
                                'Benign')
train_malignant_dir = os.path.join(train_dir,
                                   'Malignant')

test_benign_dir = os.path.join(test_dir,
                               'benign')
test_malignant_dir = os.path.join(test_dir,
                                  'malignant')


In [ ]:
len(os.listdir(train_benign_dir))

In [ ]:
len(os.listdir(test_benign_dir))

In [ ]:
IMG_HEIGHT = 212
IMG_WIDTH = 212
batch_size = 32
output_classes = 2

In [ ]:
image_gen = ImageDataGenerator(
    rescale=1./255
    )

In [ ]:
train_data_gen = image_gen.flow_from_directory(
    #batch_size=batch_size,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical')

In [ ]:
test_data_gen = image_gen.flow_from_directory(
    #batch_size=batch_size,
    directory=test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical')

In [ ]:
sample_training_images, _ = next(train_data_gen)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(3, 3, figsize=(10,10))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:9])

In [ ]:
input_shape = (IMG_HEIGHT,IMG_WIDTH,3)
base_model = ResNet50(weights='imagenet', include_top=False, pooling = 'avg', input_tensor=Input(shape=input_shape))
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
prediction = Dense(output_classes, activation=tf.nn.softmax)(x)
model = Model(inputs=base_model.input,outputs=prediction)

In [ ]:
adamax_opt=Adamax(lr = 0.001)
sgd_opt = SGD(lr=1e-06, momentum=0.0, decay=0.0, nesterov=False)
rmsp_opt = RMSprop(lr=1e-04)

model.compile(optimizer= adamax_opt, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 32
epochs = 10

In [ ]:
num_benign_train = len(os.listdir(train_benign_dir))
num_malignant_train = len(os.listdir(train_malignant_dir))


num_benign_test = len(os.listdir(test_benign_dir))
num_malignant_test = len(os.listdir(test_malignant_dir))


total_train = num_benign_train + num_malignant_train 
total_test = num_benign_test + num_malignant_test 

In [ ]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=test_data_gen,
    validation_steps=total_test // batch_size,
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience = 5
        )]
    
)


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['accuracy'],
                         mode='lines+markers',
                         name='Training accuracy'))
fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_accuracy'],
                         mode='lines+markers',
                         name='Validation accuracy'))
fig.update_layout(title='Accuracy',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))
fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title("Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.ylim(-0.25,1.00)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title("Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(test_data_gen, steps=50)

print ('Validation Score: ', score[0])
print ('Validation Accuracy: ',score[1])